In [1]:
#Importing Libraries
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

In [2]:
#Loading Dataset
dataset=pd.read_csv("Social_Network_Ads.csv")
dataset

,User ID,Gender,Age,EstimatedSalary,Purchased
0,15624510,Male,19,19000,0
1,15810944,Male,35,20000,0
2,15668575,Female,26,43000,0
3,15603246,Female,27,57000,0
4,15804002,Male,19,76000,0
...,...,...,...,...,...
395,15691863,Female,46,41000,1
396,15706071,Male,51,23000,1
397,15654296,Female,50,20000,1
398,15755018,Male,36,33000,0


In [3]:
#checking counts-Rows&columns
dataset.shape

(400, 5)

In [4]:
#get dummies
dataset=pd.get_dummies(dataset,dtype=int,drop_first=True)
dataset

,User ID,Age,EstimatedSalary,Purchased,Gender_Male
0,15624510,19,19000,0,1
1,15810944,35,20000,0,1
2,15668575,26,43000,0,0
3,15603246,27,57000,0,0
4,15804002,19,76000,0,1
...,...,...,...,...,...
395,15691863,46,41000,1,0
396,15706071,51,23000,1,1
397,15654296,50,20000,1,0
398,15755018,36,33000,0,1


In [5]:
#dropping the column that is not required which has no information
dataset=dataset.drop("User ID",axis=1)
dataset

,Age,EstimatedSalary,Purchased,Gender_Male
0,19,19000,0,1
1,35,20000,0,1
2,26,43000,0,0
3,27,57000,0,0
4,19,76000,0,1
...,...,...,...,...
395,46,41000,1,0
396,51,23000,1,1
397,50,20000,1,0
398,36,33000,0,1


In [6]:
#How many classes we are going to classify
dataset["Purchased"].value_counts()


Purchased
0    257
1    143
Name: count, dtype: int64

In [7]:
#spliting Independent and Dependent
Independent=dataset[["Age","EstimatedSalary","Gender_Male"]]
Dependent=dataset["Purchased"]

In [8]:
#split into training set and test
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(Independent,Dependent, test_size = 1/3, random_state = 0)

In [9]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

In [10]:
#model creation
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import GridSearchCV
param_grid = {'weights':['uniform', 'distance'],'metric':['minkowski','str'],} 
grid = GridSearchCV(KNeighborsClassifier(), param_grid, refit = True, verbose = 3,n_jobs=-1,scoring='f1_weighted') 
   
# fitting the model for grid search 
grid.fit(X_train, y_train) 

Fitting 5 folds for each of 4 candidates, totalling 20 fits


C:\Users\sswat\AppData\Local\anaconda3\Lib\site-packages\sklearn\model_selection\_validation.py:378: FitFailedWarning: 
10 fits failed out of a total of 20.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
5 fits failed with the following error:
Traceback (most recent call last):
  File "C:\Users\sswat\AppData\Local\anaconda3\Lib\site-packages\sklearn\model_selection\_validation.py", line 686, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "C:\Users\sswat\AppData\Local\anaconda3\Lib\site-packages\sklearn\neighbors\_classification.py", line 213, in fit
    self._validate_params()
  File "C:\Users\sswat\AppData\Local\anaconda3\Lib\site-packages\sklearn\base.py", line 600, in _validate_params
    validate_paramete

GridSearchCV(estimator=KNeighborsClassifier(), n_jobs=-1,
             param_grid={'metric': ['minkowski', 'str'],
                         'weights': ['uniform', 'distance']},
             scoring='f1_weighted', verbose=3)

In [12]:
grid_predictions = grid.predict(X_test)
        

In [13]:
#evaluation metrics(confusion_matrix)

from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_test, grid_predictions)
        

In [14]:
print(cm)

[[78  7]
 [ 4 45]]


In [15]:
from sklearn.metrics import classification_report
clf_report = classification_report(y_test, grid_predictions)

In [16]:
print(clf_report)

              precision    recall  f1-score   support

           0       0.95      0.92      0.93        85
           1       0.87      0.92      0.89        49

    accuracy                           0.92       134
   macro avg       0.91      0.92      0.91       134
weighted avg       0.92      0.92      0.92       134



In [17]:
re=grid.cv_results_
table=pd.DataFrame.from_dict(re)
table

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_metric,param_weights,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
0,0.005750,0.003556,0.010448,0.002306,minkowski,uniform,"{'metric': 'minkowski', 'weights': 'uniform'}",0.867478,0.906252,0.869709,0.944161,0.981233,0.913767,0.043854,1
1,0.001623,0.001026,0.004432,0.003015,minkowski,distance,"{'metric': 'minkowski', 'weights': 'distance'}",0.847141,0.886792,0.851527,0.944161,0.962264,0.898377,0.047185,2
2,0.001164,0.000721,0.000000,0.000000,str,uniform,"{'metric': 'str', 'weights': 'uniform'}",NaN,NaN,NaN,NaN,NaN,NaN,NaN,3
3,0.000444,0.000240,0.000000,0.000000,str,distance,"{'metric': 'str', 'weights': 'distance'}",NaN,NaN,NaN,NaN,NaN,NaN,NaN,3


In [18]:
#f1 score prediction

from sklearn.metrics import f1_score
f1_weighted=f1_score(y_test,grid_predictions,average='weighted')
print("The f1_weighted value for best parameter {}:".format(grid.best_params_),f1_weighted)

The f1_weighted value for best parameter {'metric': 'minkowski', 'weights': 'uniform'}: 0.9183922682195829


In [19]:
from sklearn.metrics import roc_auc_score
roc_auc_score(y_test,grid.predict_proba(X_test)[:,1])

0.9411764705882354